# GetGround DataTask

**The Data**
- GetGround currently has end-customers referred to us by partners, such as lettings agents and mortgage brokers. The customer then signs up for our service, and we pay the partner a small commission per referrals.
- **Referrals are on a company level**: a customer who signs up for five companies counts as five referrals. Five customers in one company count as one referral.
- **Partners each have consultants**, such as Joe Smith working at Lettings Agent A. The referrals are attributed to the specific consultant at a partner.
- **For referrals, the updated_at field** essentially says when the status went from pending to either disinterested or successful. 
- **Timestamps are in Unix Nano format.**
- **is_outbound** is true when we refer a customer to a partner, i.e. "upsell". In this case we send them the customer, and they pay us a commission. We haven't done this very thoroughly yet, so most referrals are inbound.
- **Our sales people work in a "key account" model**. Referrals come from partners, and a sales person typically manages partner accounts.
- We currently have sales people in the UK, Singapore and Hong Kong.

# Goal: 
Exploratory data analysis for performing initial investigations on data so as to discover patterns, spot anomalies, test hypotheses, and check assumptions with the help of summary statistics and graphical representations. 

# To check/plot:

**Referrals**
- Status of referrals by year.
- analyze latency for status updates, that is, partners that take time to consolidate referrals
- analyze partners that do not have referrals
- analyze sucssefull referrals by:
    - year, 
    - quarters, 
    - month x 
    - company id, 
    - partner type and 
    - sales people
    -country

## Imports and setting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import datetime
import squarify
#sql connection
from sqlalchemy import create_engine

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Db connection

In [ ]:
db_name = "postgres"
port = 5432
user = "postgres"
password = "postgres"
host = "localhost"

engine_template = "postgresql://{user}:{password}@{host}:{port}/{db_name}"
engine_str = engine_template.format(user=user, password=password, host=host, port=port, db_name=db_name)
print(engine_str)
engine = create_engine(engine_str)

# Analysis

### Connecting to analytics layer

In [ ]:
QUERY = 'SELECT * FROM dbt.sales_analytics_layer'
data_raw = pd.read_sql_query(QUERY, engine)
data_raw.head()

In [ ]:
data_raw.isna().sum()

In [ ]:
data_raw.dtypes

In [ ]:
data_raw.shape

In [ ]:
1882-412

In [ ]:
# fill na with 0 to apply conversions
data_raw = data_raw.fillna(0)

In [ ]:
### Filter referrals null
data = data_raw[data_raw['referral_id'] != 0]
data.shape

## Referrals by dates

In [ ]:
data.dtypes

In [ ]:
data.columns

#### Transform dates to datetime to extract dates, yar, month quarter etc

In [ ]:
data['partners_creation_date'] = pd.to_datetime(data['partners_creation_date'], utc=True, errors='coerce')
data['partners_update_date'] = pd.to_datetime(data['partners_update_date'], utc=True, errors='coerce')
data['referral_creation_date'] = pd.to_datetime(data['referral_creation_date'], utc=True, errors='coerce')
data['referral_update_date'] = pd.to_datetime(data['referral_update_date'], utc=True, errors='coerce')
#data.dtypes

In [ ]:
# Create column with year
data['partners_creation_year']= pd.DatetimeIndex(data['partners_creation_date']).year
data['partners_update_year']= pd.DatetimeIndex(data['partners_update_date']).year
data['referral_creation_year']= pd.DatetimeIndex(data['referral_creation_date']).year
data['referral_update_year']= pd.DatetimeIndex(data['referral_update_date']).year
# normalize
data['partners_creation_year']= data['partners_creation_year'].astype(int)
data['partners_update_year']= data['partners_update_year'].astype(int)
data['referral_creation_year']= data['referral_creation_year'].astype(int)
data['referral_update_year']= data['referral_update_year'].astype(int)

# Create column with month
data['partners_creation_month']= pd.DatetimeIndex(data['partners_creation_date']).month
data['partners_update_month']= pd.DatetimeIndex(data['partners_update_date']).month
data['referral_creation_month']= pd.DatetimeIndex(data['referral_creation_date']).month
data['referral_update_month']= pd.DatetimeIndex(data['referral_update_date']).month
# normalize
data['partners_creation_month']= data['partners_creation_month'].astype(int)
data['partners_update_month']= data['partners_update_month'].astype(int)
data['referral_creation_month']= data['referral_creation_month'].astype(int)
data['referral_update_month']= data['referral_update_month'].astype(int)

# Add quarter
data['partners_creation_quarter']= pd.DatetimeIndex(data['partners_creation_date']).quarter
data['partners_update_quarter']= pd.DatetimeIndex(data['partners_update_date']).quarter
data['referral_creation_quarter']= pd.DatetimeIndex(data['referral_creation_date']).quarter
data['referral_update_quarter']= pd.DatetimeIndex(data['referral_update_date']).quarter
# normalize
data['partners_creation_quarter']= data['partners_creation_quarter'].astype(int)
data['partners_update_quarter']= data['partners_update_quarter'].astype(int)
data['referral_creation_quarter']= data['referral_creation_quarter'].astype(int)
data['referral_update_quarter']= data['referral_update_quarter'].astype(int)

data.head()

In [ ]:
# Check year
partners_creation_year_list = list(data['partners_creation_year'].unique())
partners_update_year_list = list(data['partners_update_year'].unique())
referral_creation_year_list  = list(data['referral_creation_year'].unique())
referral_update_year_list  = list(data['referral_update_year'].unique())

partners_creation_year_list.sort()
partners_update_year_list.sort()
referral_creation_year_list.sort()
referral_update_year_list.sort() 

print('Partners creation year:', partners_creation_year_list)
print('Partners update year:', partners_update_year_list )
print('Ref creation year:', referral_creation_year_list)
print('Ref update year:', referral_update_year_list)

In [ ]:
# Check months
partners_creation_month_list = list(data['partners_creation_month'].unique())
partners_update_month_list = list(data['partners_update_month'].unique())
referral_creation_month_list  = list(data['referral_creation_month'].unique())
referral_update_month_list  = list(data['referral_update_month'].unique())

partners_creation_month_list.sort()
partners_update_month_list.sort()
referral_creation_month_list.sort()
referral_update_month_list.sort() 

print('Partners creation month:', partners_creation_month_list)
print('Partners update month:', partners_update_month_list )
print('Ref creation month:', referral_creation_month_list)
print('Ref update month:', referral_update_month_list)

In [ ]:
# Check quarters
partners_creation_quarter_list = list(data['partners_creation_quarter'].unique())
partners_update_quarter_list = list(data['partners_update_quarter'].unique())
referral_creation_quarter_list  = list(data['referral_creation_quarter'].unique())
referral_update_quarter_list  = list(data['referral_update_quarter'].unique())

partners_creation_quarter_list.sort()
partners_update_quarter_list.sort()
referral_creation_quarter_list.sort()
referral_update_quarter_list.sort() 

print('Partners creation mquarter', partners_creation_quarter_list)
print('Partners update quarter:', partners_update_quarter_list )
print('Ref creation quarter:', referral_creation_quarter_list)
print('Ref update quarter:', referral_update_quarter_list)

#### Notes dates

In [ ]:
data.referral_update_date.min()

In [ ]:
data.referral_creation_date.max()

- We do not have parters register for june and july. Why?
- What are the months with the most registrations for both partners and referrals?
- What are the months with the most referral updates?
- We have data from 2020 september to 2021 may
- We have data for all quartes

## Total Referral trend

In [ ]:
total_referrals = data.groupby(['referral_update_year', 'referral_update_month', 'status'])[['referral_id']].count().reset_index()
total_referrals  = total_referrals.sort_values(by=['referral_id'], ascending=False)
total_referrals.head()

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_month", y="referral_id", hue="referral_update_year", data=total_referrals, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Month", size=14)

# Setting the label for y-axis
plt.ylabel("Successful Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by month", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')

plt.legend(title='Year', loc='upper left')

# Finally showing the plot
plt.show()

#### Notes:
from the last two quarters of 2020 to the first two of 2021, the number of referrals registered in the system doubled in size.

## Referrals by status and year year

#### Grouping to summarize and plot

In [ ]:
referrals_by_year = data.groupby(['status', 'referral_update_year'])[['referral_id']].count().reset_index()

referrals_by_year.head()

## Referrals by year

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_year", y="referral_id", hue="status", data=referrals_by_year, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Year", size=14)

# Setting the label for y-axis
plt.ylabel("Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by year", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')


# Finally showing the plot
plt.show()

Questions: The pending status has increased a lot, are there more referrals to process and therefore the delay? What is the reason?

## Referrals by quarter

In [ ]:
referrals_by_quarter = data.groupby(['status', 'referral_update_quarter', 'referral_update_year'])[['referral_id']].count().reset_index()
referrals_by_quarter

In [ ]:
df20 = referrals_by_quarter[referrals_by_quarter['referral_update_year']==2020]
df20.head()

In [ ]:
df21 = referrals_by_quarter[referrals_by_quarter['referral_update_year']==2021]
df21.head()

### Plot ref by year and quarter

In [ ]:
# 2021
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_quarter", y="referral_id", hue="status", data=df21, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Year", size=14)

# Setting the label for y-axis
plt.ylabel("Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by quarter 2021", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')


# Finally showing the plot
plt.show()

In [ ]:
# 2021
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_quarter", y="referral_id", hue="status", data=df21, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Year", size=14)

# Setting the label for y-axis
plt.ylabel("Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by quarter 2021", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')


# Finally showing the plot
plt.show()

### Plot by quarter for 2020/2021

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_quarter", y="referral_id", hue="status", data=referrals_by_quarter, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Quarter", size=14)

# Setting the label for y-axis
plt.ylabel("Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by quarter 2021 | 2020", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')


# Finally showing the plot
plt.show()

In [ ]:
data.sort_values(by=['status'])
data.head(2)

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

g = sns.catplot(x="referral_update_quarter", hue="status", col="referral_update_year",
                data=data, kind="count",
                palette="Spectral_r",
                height=6, aspect=.7);


g.set_axis_labels("Quarter", "nº of referrals")
g.set_titles(col_template="{col_name}", row_template="{row_name}")


plt.show()

### Only Sucsessfull referrals by month and year

In [ ]:
successful_referrals = data.groupby(['status', 'referral_update_quarter', 'referral_update_year',  'referral_update_month',])[['referral_id']].count().reset_index()
successful_referrals = successful_referrals[successful_referrals['status']=='successful']
successful_referrals = successful_referrals.sort_values(by=['referral_update_year'])
successful_referrals

In [ ]:
successful_referrals.dtypes

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

# Defining the values for x-axis, y-axis and from which dataframe the values are to be picked
plot = sns.barplot(x="referral_update_month", y="referral_id", hue="referral_update_year", data=successful_referrals, palette="Spectral")

# Setting the label for x-axis
plt.xlabel("Month", size=14)

# Setting the label for y-axis
plt.ylabel("Successful Referrals", size=14)

# Setting the title for the graph
plt.title("Referrals by company", size=16)

# Iterrating over the bars one-by-one
for bar in plot.patches:
    plot.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=12, xytext=(0, 8),
                   textcoords='offset points')

plt.legend(title='Year', loc='upper left')

# Finally showing the plot
plt.show()

#### **Notes**:
- the month with the highest number of successful referrals was: October - 2021, with an ammount of 168 successful referrals.
- the month with the lowest number of success in referrals was May -2020, BECAUSE WE ONLY HAVE DATA UNTILL MAY 5TH.

## Referrals by company
- **Referrals are on a company level**: a customer who signs up for five companies counts as five referrals. Five customers in one company count as one referral.

#### Prepare the data

In [ ]:
#data.columns

In [ ]:
len(data.company_id.unique())

In [ ]:
data['company_id'] = data.company_id.astype(int)
#data.dtypes

 We have 776 companies with referrals updated

In [ ]:
referrals_by_company = data.groupby(['status', 'company_id'])[['referral_id']].count().reset_index()
#referrals_by_company = referrals_by_company[(referrals_by_company['company_id']!= 0) & (referrals_by_company['referral_id'] >= 4)]
referrals_by_company = referrals_by_company[(referrals_by_company['company_id']!= 0) & (referrals_by_company['status']=='successful') & (referrals_by_company['referral_id'] >= 2)]
referrals_by_company = referrals_by_company.sort_values(by=['referral_id'], ascending=False)
referrals_by_company.head()

top companies: 809, 811, 1124, 1102

In [ ]:
referrals_by_company.shape

In [ ]:
len(referrals_by_company.company_id.unique())

Há companhias que tem mais de uma referral atrelada!

In [ ]:
referrals_by_company.referral_id.unique()

In [ ]:
referrals_by_company.status.unique()

#### unmelt to create a horizontal stacked bar plot

In [ ]:

# unmelting
reshaped_df = referrals_by_company.pivot(index='company_id', columns='status').reset_index()
  
# displaying the reshaped data frame
reshaped_df.head()

In [ ]:
# remove extra level
reshaped_df = reshaped_df.droplevel(0, axis=1) 
reshaped_df.head()

In [ ]:
# rename col
reshaped_df = reshaped_df.rename(columns={'':'company_id'})

In [ ]:
reshaped_df.isnull().sum()

In [ ]:
reshaped_df = reshaped_df.fillna(0)
reshaped_df.head()

In [ ]:
reshaped_df.columns

In [ ]:
reshaped_df['total'] = reshaped_df['disinterested'] + reshaped_df['pending'] + reshaped_df['successful']
reshaped_df.head()

## Referrals inbound x outbound per year and country

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
out_x_in = data.groupby(['is_outbound'])[['referral_id']].count().reset_index()
print(out_x_in.referral_id.sum())
out_x_in

####  Plot composition 

In [ ]:
# Define data
data = list(out_x_in.referral_id.unique())
labels = ['Inbound', 'outbound']

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

#define Seaborn color palette to use
#colors = sns.color_palette('pastel')[0:5]
colors= ['teal', 'tan']
#create pie chart
plt.pie(data, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

##### Note: 
- We have 932 inbound referrals and 538 inbound referrals and 538 outbound
- i.e 63% of our referrals are inbound, however we have room to increase and invest in outbound, that represent 37%  of the referrals.


In [ ]:
outbound = data.groupby(['status', 'company_id', 'country'])[['is_outbound']].sum().reset_index()
outbound = outbound[(outbound['is_outbound']!=0) & (outbound['company_id']!= 0) & (outbound['country']!= 0)]
outbound = outbound.sort_values(by=['is_outbound'], ascending=False)
outbound.shape

In [ ]:
outbound.country.unique()

In [ ]:
sing = outbound[outbound['country']=='Singapore']
sing.shape

In [ ]:
sing.head()

In [ ]:
outbound = data.groupby(['status', 'company_id', 'country'])[['is_outbound']].sum().reset_index()
outbound = outbound[(outbound['is_outbound']!=0) & (outbound['company_id']!= 0)  & (outbound['is_outbound']>3)]
outbound = outbound.sort_values(by=['is_outbound'], ascending=False)
outbound.shape

In [ ]:
outbound.is_outbound.unique()

In [ ]:
outbound.head()

In [ ]:
outbound.country.unique()

In [ ]:
# em qual mes tivemos maior numero de outbound
outbound_year = data.groupby(['status', 'company_id', 'country', 'referral_update_year'])[['is_outbound']].sum().reset_index()
outbound_year = outbound[(outbound['is_outbound']!=0) & (outbound['company_id']!= 0)  & (outbound['is_outbound']>3)]
outbound_year = outbound.sort_values(by=['is_outbound'], ascending=False)
outbound_year

#### Notes:
- Em Uk temos mais outbound, faz sentido é melhor começar por lá
- temos um total de 340 registros de vendas outbound
-  Temos 19 empresas com outbound maior que , todas em UK
- Companhias com maior numero referral outbound: 1057
- Hã 70 ocorrencias de servicos upsell em singapore, pode ser um futuro mercado para expansao deste produto
- As tres empresas com maior numero de outbound sao: 1053, 976, 952

In [ ]:
# empresas fazendo outbound

# paises onde outbound esta crescendo

In [ ]:
#outbound_ref.referral_id.unique()

## Referrals density by status

#### Prepare the data to plot areas

In [ ]:
referrals_by_status = data.groupby(['status', 'referral_update_year'])[['referral_id']].count().reset_index()
# unmelting
reshaped_df_2 = referrals_by_status.pivot(index='referral_update_year', columns='status').reset_index()
# remove extra level
reshaped_df_2 = reshaped_df_2.droplevel(0, axis=1) 
# rename col
reshaped_df_2 = reshaped_df_2.rename(columns={'':'year'})
reshaped_df_2 = reshaped_df_2.fillna(0)
reshaped_df_2.head()

#### Plot status area chart

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

#colormap
colormap=['tan','teal','darkseagreen']

#plot
plt.stackplot(reshaped_df_2.year, reshaped_df_2.pending, reshaped_df_2.disinterested, reshaped_df_2.successful,
             labels=['pending', 'disinterested', 'successful'], colors=colormap)

#add legend
plt.legend(title='Year', loc='upper left')

#add axis labels
plt.xlabel('Month')
plt.ylabel('Referrals')

# Setting the title for the graph
plt.title("Referrals by status", size=16)


#display area chart
plt.show()

In [ ]:
total_referrals_status = data.groupby(['status'])[['referral_id']].count().reset_index()
total_referrals_status  = total_referrals_status.sort_values(by=['referral_id'], ascending=False)
total_referrals_status

##### Notes: No total temos 878 referrals com status successful, 535 pending  and 57 disinterested

#### Plot the distribution of referrals, conditional on status

In [ ]:
sns.set_theme(style="whitegrid")
sns.displot(
    data=total_referrals,
    x="referral_id", hue="status",
    kind="kde", height=6,
    multiple="fill", clip=(0, None),
    palette="ch:rot=-.25,hue=1,light=.75",
)

In [ ]:
# Referrals by consultant
# how many consultant do we have?
# 

## Referrals by country

In [ ]:
countries = data.groupby(['country'])[['referral_id']].count().reset_index()
countries = countries[countries['country']!=0]
print(countries.referral_id.sum())
countries = countries.sort_values(by=['referral_id'], ascending=False)
countries

In [ ]:
#### Notes: The country with more referrals is UK, as expected. And the second prosperous country is singapore

####  Plot composition 

In [ ]:
# Define data
data2 = list(countries.referral_id.unique())
data2

In [ ]:
# Define labels
labels = list(countries.country.unique())
labels

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(10, 6))

#define Seaborn color palette to use
#colors = sns.color_palette('pastel')[0:5]
colors= ['teal', 'tan', 'darkseagreen']
#create pie chart
plt.pie(data2, labels = labels, colors = colors, autopct='%.0f%%')
plt.show()

## Referrals by partner type

In [ ]:
partners = data.groupby(['partner_type'])[['referral_id']].count().reset_index()
partners = partners[partners['partner_type']!=0]
print(partners.referral_id.sum())
partners = partners.sort_values(by=['referral_id'], ascending=False)
partners

#### Plot partners type composition

In [ ]:
# Define the style
sns.set_theme(style="whitegrid")

# Define the plot size
plt.figure(figsize=(12, 6))

# define cmap

# create a color palette, mapped to these values
#cmap = matplotlib.cm.Set2
cmap = matplotlib.cm.Paired_r
mini=partners.referral_id.min()
maxi=partners.referral_id.max()
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in partners.referral_id]

# plot it
squarify.plot(sizes=partners['referral_id'], label=partners['partner_type'], color=colors, alpha=.8)
plt.axis('off')
plt.show()

### Top sales people

# Results

limitations: we do not have a complete darange for the 2 years to analyze the trend by month or quarter, wich means, we cannot confirm that there is a componente of sazonality. Just that the referrals increased from 2020 to 2021.
As comapnias com mais referrals bem sucedidas atreladas nao possuem company id, pq na verdade é o count para nulkl data
# o numero maximo de referrals por companhia é 6

Of course, it is important to keep in mind that these correlations do not indicate causal relationships, or even the direction of the relationship, between the variables examined. These are just starting points for further investigation, but interesting ones nonetheless.
inbound x outbound
onde o sistema upseel pode virar novo produto
poderiamos investir em um processo de discovery para entender a categoria outros.
O que significa IFA?

### 1. 